In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/My Drive/Colab Notebooks/medical image/

Mounted at /content/drive


In [ ]:
!nvidia-smi 

Sun Mar 20 13:56:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np

#Dataset

In [ ]:
from torch.utils.data import Dataset
from pathlib import Path
from PIL import Image


class IMAGE_Dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.x = []
        self.y = []
        self.transform = transform
        self.num_classes = 0
        #print(self.root_dir.name)
        for i, _dir in enumerate(self.root_dir.glob('*')):
            for file in _dir.glob('*'):
                self.x.append(file)
                self.y.append(i)

            self.num_classes += 1
            #print(self.num_classes)
        #print(self.num_classes)
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        image = Image.open(self.x[index]).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, self.y[index]


#VGG16

In [ ]:
import torch.nn as nn
import math

X = 256

class VGG16(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            # input shape: (batch_size, 3, 224, 224) and
            # downsampled by a factor of 2^5 = 32 (5 times maxpooling)
            # So features' shape is (batch_size, 7, 7, 512)
            nn.Linear(in_features=7 * 7 * 512, out_features=X),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=X, out_features=X),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=X, out_features=num_classes)
        )

        # initialize parameters
        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                n = module.kernel_size[0] * module.kernel_size[1] * module.out_channels
                module.weight.data.normal_(0, math.sqrt(2. / n))
                module.bias.data.zero_()
            elif isinstance(module, nn.Linear):
                module.weight.data.normal_(0, 0.01)
                module.bias.data.zero_()

    def forward(self, x):
        x = self.features(x)
        # flatten
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [ ]:
class Block(nn.Module):
  def __init__(self,in_channels,out_channels,stride=1):
    super(Block,self).__init__()
    self.plain=nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_channels)
    )
    #判斷strides是否等於1，輸出入channel是否相等，否就做downsample使用1x1convolution(stride=2=>3x3=>1x1)
    if stride!=1 or in_channels!=out_channels: 
      self.shortcut=nn.Sequential(
          nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
          nn.BatchNorm2d(out_channels)
      )
    self.short_cut=(stride!=1) or (in_channels!=out_channels)
  def forward(self,inputs):
    x=self.plain(inputs)
    if not self.short_cut:
      shortcut=inputs
    else:
      shortcut=self.shortcut(inputs)
    # print(x.shape)
    # print(shortcut.shape)
    return x+shortcut

class ResNet(nn.Module):
  def __init__(self):
    super(ResNet, self).__init__()
    self.model=nn.Sequential(
        Block(3,64),
        Block(64,64),
        Block(64,256,2),

        Block(256,128),
        Block(128,128),
        Block(128,512,2),

        Block(512,256),
        Block(256,256),
        Block(256,1024,2),

        Block(1024,512),
        Block(512,512),
        Block(512,2048,2),

        nn.AdaptiveAvgPool2d((1,1)),
        nn.Flatten(),
        nn.Linear(2048,2048),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(2048,2048),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(2048, 2)
    )
  def forward(self,inputs):
    # x=self.block1(inputs)
    y=self.model(inputs)
    return y

#Train

In [ ]:
import torch
import torch.nn as nn
# from Model_VGG16 import VGG16
# from dataset import IMAGE_Dataset
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from pathlib import Path
import copy
import time
import os
import torchvision.models as models
##REPRODUCIBILITY
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#args = parse_args()
#CUDA_DEVICES = args.cuda_devices
#DATASET_ROOT = args.path
CUDA_DEVICES = 0
DATASET_ROOT = '/content/drive/My Drive/colab_network/medical image/train/'
DATASET_valROOT = '/content/drive/My Drive/colab_network/medical image/val/'
PATH_TO_WEIGHTS = '/content/model-0.80-best_train_acc.pth' # Your model name

# Initial learning rate
init_lr = 0.01

# Save model every 5 epochs
checkpoint_interval = 5
if not os.path.isdir('/content/drive/My Drive/colab_network/medical_image/Checkpoint/'):
    os.mkdir('/content/drive/My Drive/colab_network/medical_image/Checkpoint/')


# Setting learning rate operation
def adjust_lr(optimizer, epoch):
    
    # 1/10 learning rate every 5 epochs
    lr = init_lr * (0.1 ** (epoch // 5))
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def train():
    data_transform = transforms.Compose([
        transforms.Resize((224,224)),
        # transforms.RandomCrop(224, pad_if_needed=True),
        # transforms.RandomRotation(2.8),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    #print(DATASET_ROOT)
    train_set = IMAGE_Dataset(Path(DATASET_ROOT), data_transform)
    
    # If out of memory , adjusting the batch size smaller
    data_loader = DataLoader(dataset=train_set, batch_size=16, shuffle=True, num_workers=0)#16改128

    val_set = IMAGE_Dataset(Path(DATASET_valROOT), data_transform)
    val_data_loader = DataLoader(dataset=val_set, batch_size=16, shuffle=True, num_workers=0)
    classes = [_dir.name for _dir in Path(DATASET_valROOT).glob('*')]
    classes.sort()
    classes.sort(key = len)
    #print(train_set.num_classes)
    # model = VGG16(num_classes=train_set.num_classes)
    # model = ResNet()
    #model=models.shufflenet_v2_x1_0(pretrained=True)
    num_classes = 2
    
    model = models.resnet50(pretrained=True)

    num_features = model.fc.out_features
    model.fc = nn.Linear(2048,2)
    #model.add_module("add_dropout",nn.Dropout())
    #model.add_module("add_fc",nn.Linear(num_features, num_classes))
    #model = models.vgg19(pretrained=True)
    #model.classifier[6] = nn.Linear(in_features=4096, out_features=2, bias=True)
    model = model.cuda(CUDA_DEVICES)
    
    #para_optim = []
    #for i, single_layer in  enumerate(model.modules()):
    #    print(i, single_layer)
    #if i > 50:      # 前面37层冻结
    #  for param in single_layer.parameters():
    #         para_optim.append(param)
    #  else:           # 后面7层不冻结正常更新
    #     for param in single_layer.parameters():
    #         param.requires_grad = False
    #print(f'para_optim len = {len(para_optim)}')
    print(model)
    model.train()

    best_model_params = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # Training epochs
    num_epochs = 20 #20改3
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer setting
    #optimizer = torch.optim.SGD(params=model.parameters(), lr=init_lr, momentum=0.9)
    #optimizer = torch.optim.RMSprop(params=model.parameters(), lr=init_lr, alpha=0.9)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=init_lr)
    # Log 
    with open('TrainingAccuracy.txt','w') as fAcc:
        print('Accuracy\n', file = fAcc)
    with open('TrainingLoss.txt','w') as fLoss:
        print('Loss\n', file = fLoss)

    for epoch in range(num_epochs):
        model.train()
        localtime = time.asctime( time.localtime(time.time()) )
        print('Epoch: {}/{} --- < Starting Time : {} >'.format(epoch + 1,num_epochs,localtime))
        print('-' * len('Epoch: {}/{} --- < Starting Time : {} >'.format(epoch + 1,num_epochs,localtime)))

        training_loss = 0.0
        training_corrects = 0
        adjust_lr(optimizer, epoch)

        for i, (inputs, labels) in enumerate(data_loader):
            inputs = Variable(inputs.cuda(CUDA_DEVICES))
            labels = Variable(labels.cuda(CUDA_DEVICES))

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            training_loss += float(loss.item() * inputs.size(0))
            training_corrects += torch.sum(preds == labels.data)
        training_loss = training_loss / len(train_set)
        training_acc = training_corrects.double() /len(train_set)
        print('Training loss: {:.4f}\taccuracy: {:.4f}\n'.format(training_loss,training_acc))
        

        # Check best accuracy model ( but not the best on test )
        if training_acc > best_acc:
            best_acc = training_acc
            best_model_params = copy.deepcopy(model.state_dict())


        with open('TrainingAccuracy.txt','a') as fAcc:
            print('{:.4f} '.format(training_acc), file = fAcc)
        with open('TrainingLoss.txt','a') as fLoss:
            print('{:.4f} '.format(training_loss), file = fLoss)

        # Checkpoint
        if (epoch + 1) % checkpoint_interval == 0:
            torch.save(model, '/content/drive/My Drive/colab_network/medical_image/M-model-epoch-{:d}-train.pth'.format(epoch + 1))
        # DATASET_valROOT = '/content/drive/My Drive/Colab Notebooks/medical image/test/'
    # PATH_TO_WEIGHTS = '/content/model-0.54-best_train_acc.pth' # Your model name

    

    # Load model
    # model = torch.load(PATH_TO_WEIGHTS)
    # model = model.cuda(CUDA_DEVICES)
        model.eval()
        total_correct = 0
        total = 0
        class_correct = list(0. for i in enumerate(classes))
        class_total = list(0. for i in enumerate(classes))

        with torch.no_grad():
            for inputs, labels in val_data_loader:
                inputs = Variable(inputs.cuda(CUDA_DEVICES))
                labels = Variable(labels.cuda(CUDA_DEVICES))
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                
                # totoal
                total += labels.size(0)
                total_correct += (predicted == labels).sum().item()
                c = (predicted == labels).squeeze()
                
                # batch size
                for i in range(labels.size(0)):
                    label = labels[i]
                    class_correct[label] += c[i].item()
                    class_total[label] += 1

            for i, c in enumerate(classes):
                print('Accuracy of %5s : %8.4f %%' % (
                c, 100 * class_correct[i] / class_total[i]))

            # Accuracy
            print('\nAccuracy on the ALL test images: %.4f %%'
              % (100 * total_correct / total))
    # Save best training/valid accuracy model ( not the best on test )
    model.load_state_dict(best_model_params)
    best_model_name = '/content/drive/My Drive/colab_network/medical_image/M-model-{:.2f}-best_train_acc.pth'.format(best_acc)
    torch.save(model, best_model_name)
    print("Best model name : " + best_model_name)
    total = sum([param.nelement() for param in model.parameters()])
    print("Number of parameter: %.2fM" % (total/1e6))


if __name__ == '__main__':
    train()
    


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

#val

In [ ]:
import torch
# from utils import parse_args
from torch.autograd import Variable
from torchvision import transforms
from pathlib import Path
from PIL import Image
from torch.utils.data import DataLoader
# from dataset import IMAGE_Dataset
import numpy as np

CUDA_DEVICES = 0
DATASET_valROOT = '/content/drive/My Drive/colab_network/medical image/val/'
PATH_TO_WEIGHTS = '/content/model-0.54-best_train_acc.pth' # Your model name


def val():
    CUDA_DEVICES = 0
    #DATASET_valROOT = '/content/drive/My Drive/colab_network/medical image/test/'
    # PATH_TO_WEIGHTS = '/content/model-0.54-best_train_acc.pth' # Your model name

    data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                             0.229, 0.224, 0.225])
    ])
    val_set = IMAGE_Dataset(Path(DATASET_valROOT), data_transform)
    val_data_loader = DataLoader(
        dataset=val_set, batch_size=32, shuffle=True, num_workers=0)
    classes = [_dir.name for _dir in Path(DATASET_valROOT).glob('*')]
    classes.sort()
    classes.sort(key = len)

    # Load model
    model = torch.load(PATH_TO_WEIGHTS)
    model = model.cuda(CUDA_DEVICES)
    model.eval()
    total_correct = 0
    total = 0
    class_correct = list(0. for i in enumerate(classes))
    class_total = list(0. for i in enumerate(classes))

    with torch.no_grad():
        for inputs, labels in val_data_loader:
            inputs = Variable(inputs.cuda(CUDA_DEVICES))
            labels = Variable(labels.cuda(CUDA_DEVICES))
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            # totoal
            total += labels.size(0)
            total_correct += (predicted == labels).sum().item()
            c = (predicted == labels).squeeze()
            
            # batch size
            for i in range(labels.size(0)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i, c in enumerate(classes):
        print('Accuracy of %5s : %8.4f %%' % (
        c, 100 * class_correct[i] / class_total[i]))

    # Accuracy
    print('\nAccuracy on the ALL test images: %.4f %%'
      % (100 * total_correct / total))



if __name__ == '__main__':
    val()

#test

In [ ]:
import torch
# from utils import parse_args
from torch.autograd import Variable
from torchvision import transforms
from pathlib import Path
from PIL import Image
from torch.utils.data import DataLoader
# from dataset import IMAGE_Dataset
import numpy as np

CUDA_DEVICES = 0
DATASET_ROOT = '/content/drive/My Drive/colab_network/medical image/test/'
PATH_TO_WEIGHTS = '/content/drive/My Drive/colab_network/medical_image/M-model-0.69-best_train_acc.pth' # Your model name


def test():
    data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                             0.229, 0.224, 0.225])
    ])
    test_set = IMAGE_Dataset(Path(DATASET_ROOT), data_transform)
    data_loader = DataLoader(
        dataset=test_set, batch_size=32, shuffle=True, num_workers=0)
    classes = [_dir.name for _dir in Path(DATASET_ROOT).glob('*')]
    classes.sort()
    classes.sort(key = len)

    # Load model
    model = torch.load(PATH_TO_WEIGHTS)
    model = model.cuda(CUDA_DEVICES)
    model.eval()
    

    total_correct = 0
    total = 0
    class_correct = list(0. for i in enumerate(classes))
    class_total = list(0. for i in enumerate(classes))

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = Variable(inputs.cuda(CUDA_DEVICES))
            labels = Variable(labels.cuda(CUDA_DEVICES))
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            # totoal
            total += labels.size(0)
            total_correct += (predicted == labels).sum().item()
            c = (predicted == labels).squeeze()
            
            # batch size
            for i in range(labels.size(0)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i, c in enumerate(classes):
        print('Accuracy of %5s : %8.4f %%' % (
        c, 100 * class_correct[i] / class_total[i]))

    # Accuracy
    print('\nAccuracy on the ALL test images: %.4f %%'
      % (100 * total_correct / total))



if __name__ == '__main__':
    test()

Accuracy of   cnv :  54.6012 %
Accuracy of   pcv :  74.5098 %

Accuracy on the ALL test images: 64.2405 %


#util(不用跑~)

In [ ]:
import argparse

def parse_args():
    desc = 'PyTorch example code for Kaggle competition -- Plant Seedlings Classification.\n' \
           'See https://www.kaggle.com/c/plant-seedlings-classification'
    parser = argparse.ArgumentParser(description=desc)
    parser.add_argument('-p', '--path', help='path to dataset')
    parser.add_argument('-w', '--weight', help='path to model weights')
    parser.add_argument('-c', '--cuda_devices', type=int, help='path to model weights')
    return parser.parse_args()